In [1]:
import numpy as np
from libsvm.svmutil import *
from numpy import matmul as mul

## LoadData

In [2]:
def LoadData(X_path, Y_path):
    X = np.loadtxt(X_path, delimiter=',') 
    Y = np.loadtxt(Y_path, delimiter=',') 
    return X, Y

In [3]:
X_train, Y_train = LoadData("./data/X_train.csv", "./data/Y_train.csv")
X_test, Y_test = LoadData("./data/X_test.csv", "./data/Y_test.csv")

## Different kernel
-t, kernel_type :
* 0 -> linear
* 1 -> polynomial
* 2 -> RBF

In [4]:
# task 1 different kernrl
kernels = {'linear': 0, 'polynomial': 1, 'RBF': 2}

for key, value in kernels.items():
    print(f"kernel: {key}", end = ", ")
    model = svm_train(Y_train, X_train, f"-t {value}")
    pred = svm_predict(Y_test, X_test, model)


kernel: linear, Accuracy = 95.08% (2377/2500) (classification)
kernel: polynomial, Accuracy = 34.68% (867/2500) (classification)
kernel: RBF, Accuracy = 95.32% (2383/2500) (classification)


## Grid search
| arg | useage| linear | polynomial | RBF | default value|
|--|--|--|--|--|--|
|-c| cost for C-SVC| v | v | v |1|
|-g |gamma| | v | v | 1/number of features|
|-d |degree| |v |  | 3|
|-r |coefficient 0| |v |  | 0|
|-v| n-fold | v | v | v ||


In [27]:
def GridSearch(X,Y):    
    D = [2, 3, 4]
    Gamma = [1e-2, 1e-1, 1] 
    Coef0 = [2, 4, 10]
    C = [0.001, 0.01, 0.1, 1, 10]
    v = 5
    
    for key, kernel in kernels.items():
        print(f"\nkernel: {key}")
        print("Default:", end = " ")
        res = svm_train(Y, X, f"-t {kernel} -v {v}")
        if kernel==0:
            for c in C:
                arg = f"-t {kernel} -v {v} -c {c}"
                print(f"-c {c:<5.3f}", end = " ") 
                res = svm_train(Y, X, arg) 
        elif kernel==1:
            for c in C:
                for d in D:
                    for gamma in Gamma:
                        for coef0 in Coef0:
                            arg = f"-t {kernel} -v {v} -c {c} -d {d} -g {gamma} -r {coef0} "
                            print(f"-c {c:<5.3f} -d {d:<1d} -g {gamma:<3.2f} -r {coef0:<2d} ", end = " ") 
                            res = svm_train(Y, X, arg)              
        else:
            for c in C:
                for gamma in Gamma:
                    arg = f"-t {kernel} -v {v} -c {c} -g {gamma}"
                    print(f"-c {c:<5.3f} -g {gamma:<3.2f}", end = " ") 
                    res = svm_train(Y, X, arg)   




In [28]:
# part 2 grid search
kernels = {'linear': 0, 'polynomial': 1, 'RBF': 2}
GridSearch(X_train, Y_train)


kernel: linear
Default: Cross Validation Accuracy = 96.24%
-c 0.001 Cross Validation Accuracy = 95.52%
-c 0.010 Cross Validation Accuracy = 97.12%
-c 0.100 Cross Validation Accuracy = 96.84%
-c 1.000 Cross Validation Accuracy = 96.36%
-c 10.000 Cross Validation Accuracy = 96%

kernel: polynomial
Default: Cross Validation Accuracy = 32.56%
-c 0.001 -d 2 -g 0.01 -r 2   Cross Validation Accuracy = 80.26%
-c 0.001 -d 2 -g 0.01 -r 4   Cross Validation Accuracy = 88.14%
-c 0.001 -d 2 -g 0.01 -r 10  Cross Validation Accuracy = 92.58%
-c 0.001 -d 2 -g 0.10 -r 2   Cross Validation Accuracy = 95.8%
-c 0.001 -d 2 -g 0.10 -r 4   Cross Validation Accuracy = 96.16%
-c 0.001 -d 2 -g 0.10 -r 10  Cross Validation Accuracy = 96.54%
-c 0.001 -d 2 -g 1.00 -r 2   Cross Validation Accuracy = 98.16%
-c 0.001 -d 2 -g 1.00 -r 4   Cross Validation Accuracy = 98.2%
-c 0.001 -d 2 -g 1.00 -r 10  Cross Validation Accuracy = 98.06%
-c 0.001 -d 3 -g 0.01 -r 2   Cross Validation Accuracy = 91.28%
-c 0.001 -d 3 -g 0.0

In [5]:
# Linear best parameter: -c 0.01
# Polynomial best parameter: -c 0.1 -d 2 -g 1 -r 10
# RBF best parameter: -c 10 -g 0.01

kernels = {'linear': 0, 'polynomial': 1, 'RBF': 2}
best_opt = {'linear': "-c 0.01",
            'polynomial': "-c 0.1 -d 2 -g 1 -r 10",
            'RBF': "-c 10 -g 0.01"}

for key, value in kernels.items():
    print(f"kernel: {key}", end = ", ")
    print(f"best opt: -t {value} {best_opt[key]}")
    model = svm_train(Y_train, X_train, f"-t {value} "+best_opt[key])
    pred = svm_predict(Y_test, X_test, model)
    print("")

kernel: linear, best opt: -t 0 -c 0.01
Accuracy = 95.96% (2399/2500) (classification)

kernel: polynomial, best opt: -t 1 -c 0.1 -d 2 -g 1 -r 10
Accuracy = 97.84% (2446/2500) (classification)

kernel: RBF, best opt: -t 2 -c 10 -g 0.01
Accuracy = 98.2% (2455/2500) (classification)



## linear + RBF

| Kernel|| $$ K_{ij} $$|| $$ Matrix　K $$|
|--|--|--|--|--|
|Linear|| $$ K_{ij} = x_i \cdot x_j $$|| $$ K = X X^{'T} $$|
|RBF|| $$ K_{ij} = \exp^{-\gamma \|x_i - x_j\|^2} $$|| $$ K = e^{-\gamma D},  where  D = \|x_i\|^2 + \|x_j\|^2 - 2 (x_i \cdot x_j) $$ |


In [6]:
def Linear_K(X1,X2):
    return mul(X1,X2.T)

In [7]:
def RBF_K(X1,X2, gamma):
    square_X1 = np.sum(X1 ** 2, axis=1).reshape(-1, 1)
    square_X2 = np.sum(X2 ** 2, axis=1)
    d = square_X1 - 2 * mul(X1, X2.T) + square_X2
    k = np.exp(-gamma* d)
    return k

Precomputed Kernel<br>
New training instance for xi: 0:i 1:K(xi,x1) ... L:K(xi,xL)<br>
the first column must be the "ID" of xi.

In [8]:
gamma = 0.01
K_train = Linear_K(X_train, X_train) + RBF_K(X_train, X_train, gamma)
K_test = Linear_K(X_train,X_test).T + RBF_K(X_train,X_test, gamma).T

# add index for every Ki
train_idx = np.arange(1,K_train.shape[0]+1)
K_train = np.column_stack((train_idx, K_train))

test_idx = np.arange(1,K_test.shape[0]+1)
K_test = np.column_stack((test_idx, K_test))

In [9]:
# -t 4: precomputed kernel
model = svm_train(Y_train, K_train, "-t 4")
pred = svm_predict(Y_test, K_test, model)

Accuracy = 95.32% (2383/2500) (classification)
